In [1]:
using Evolutionary
using Flux
using Flux: onehot, onecold, logitcrossentropy #, throttle, @epochs
using MLDatasets
using Random

We'll be using Iris dataset for this exmaple

In [2]:
features = Iris.features();
slabels = Iris.labels();
classes = unique(slabels)  # unique classes in the dataset
nclasses = length(classes) # number of classes
d, n = size(features)          # dimension and size if the dataset

(4, 150)

Convert feature and labels in appropriate for training format

In [3]:
data = [ (x, onehot(l, classes)) for (x, l) in zip(eachcol(features), slabels)]

150-element Vector{Tuple{SubArray{Float64, 1, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, Flux.OneHotArray{UInt32, 3, 0, 1, UInt32}}}:
 ([5.1, 3.5, 1.4, 0.2], [1, 0, 0])
 ([4.9, 3.0, 1.4, 0.2], [1, 0, 0])
 ([4.7, 3.2, 1.3, 0.2], [1, 0, 0])
 ([4.6, 3.1, 1.5, 0.2], [1, 0, 0])
 ([5.0, 3.6, 1.4, 0.2], [1, 0, 0])
 ([5.4, 3.9, 1.7, 0.4], [1, 0, 0])
 ([4.6, 3.4, 1.4, 0.3], [1, 0, 0])
 ([5.0, 3.4, 1.5, 0.2], [1, 0, 0])
 ([4.4, 2.9, 1.4, 0.2], [1, 0, 0])
 ([4.9, 3.1, 1.5, 0.1], [1, 0, 0])
 ([5.4, 3.7, 1.5, 0.2], [1, 0, 0])
 ([4.8, 3.4, 1.6, 0.2], [1, 0, 0])
 ([4.8, 3.0, 1.4, 0.1], [1, 0, 0])
 ⋮
 ([6.0, 3.0, 4.8, 1.8], [0, 0, 1])
 ([6.9, 3.1, 5.4, 2.1], [0, 0, 1])
 ([6.7, 3.1, 5.6, 2.4], [0, 0, 1])
 ([6.9, 3.1, 5.1, 2.3], [0, 0, 1])
 ([5.8, 2.7, 5.1, 1.9], [0, 0, 1])
 ([6.8, 3.2, 5.9, 2.3], [0, 0, 1])
 ([6.7, 3.3, 5.7, 2.5], [0, 0, 1])
 ([6.7, 3.0, 5.2, 2.3], [0, 0, 1])
 ([6.3, 2.5, 5.0, 1.9], [0, 0, 1])
 ([6.5, 3.0, 5.2, 2.0], [0, 0, 1])
 ([6.2, 3.4, 5.4, 2.3], [0, 0, 1

Define some auxiliary functions: model accuracy and its loss

In [4]:
accuracy(model,x,y) = sum(onecold(model(x)) .== onecold(y))/size(x,2)
accuracy(xy, model) = sum( onecold(model(x)) .== onecold(y) for (x,y) in xy) /length(xy)

loss(model) = (x,y)->logitcrossentropy(model(x), y)
loss(model,x,y) = loss(model)(x, y)
loss(xy, model) = loss(model)(hcat(map(first,xy)...), hcat(map(last,xy)...))

loss (generic function with 3 methods)

## Backpropagation

We use a multi-layer perceptron (MLP) model for our classification problem

In [27]:
model = Chain(Dense(d, 15,relu), Dense(15, nclasses))

Chain(
  Dense(4, 15, relu),                   # 75 parameters
  Dense(15, 3),                         # 48 parameters
)                   # Total: 4 arrays, 123 parameters, 748 bytes.

Train the model using the backpropagation method

## Genetic Algorithm

First, we specify a fitness function. We have already defined a loss function for our backpropagation model, so we are going to reuse it.

- We pass an individual to the fitness function to evaluate a loss of the MLP.
- GA optimization searches an individual to minimize the fitness function. In our case optimization direction is aligned with the backpropagation model loss function as we seek to minimize the MLP loss.

In [6]:
fitness(m) = loss(data, m)

fitness (generic function with 1 method)

In [7]:
fitness(model)

3.3645393195604965

### Genetic Operators

We need to define a crossover operator to combine the information of two parents model to generate new individuals. The objective is to increase genetic variability and provide better options.
- Flattent the MLP networks into parameter vector representations
- Perform a crossover operation on the parameter vectors
- Reconstruct MLPs from the parameter vectors

In [8]:
function uniform_mlp(m1::T, m2::T) where {T <: Chain}
    θ1, re1 = Flux.destructure(m1);
    θ2, re2 = Flux.destructure(m2);
    c1, c2 =uniform(θ1,θ2)
    return re1(c1), re2(c2)
end

uniform_mlp (generic function with 1 method)

In [9]:
uniform_mlp(model, model)

(Chain(Dense(4, 15, relu), Dense(15, 3)), Chain(Dense(4, 15, relu), Dense(15, 3)))

Similarly, we rewrite a `gaussian` mutatation operator for MLP.

In [10]:
function gaussian_mlp(σ::Real = 1.0)
    vop = gaussian(σ)
    function mutation(recombinant::T) where {T <: Chain}        
        θ, re = Flux.destructure(recombinant)
        return re(convert(Vector{Float32}, vop(θ)))
    end
    return mutation
end

gaussian_mlp (generic function with 2 methods)

In [11]:
gaussian_mlp(0.5)(model)

Chain(
  Dense(4, 15, relu),                   # 75 parameters
  Dense(15, 3),                         # 48 parameters
)                   # Total: 4 arrays, 123 parameters, 748 bytes.

### Population

We also require a function for generating a random population of the individuls required for evolutionary optimizations. Our polulation consists of MLP objects, `Flux.Chain` type.

We need to override `Evolutionary.initial_population` which will allows us to create population of the random MPL objects.

In [12]:
import Evolutionary.initial_population
function initial_population(method::M, individual::Chain) where {M<:Evolutionary.AbstractOptimizer}
    θ, re = Flux.destructure(individual);
    [re(randn(length(θ))) for i in 1:Evolutionary.population_size(method)]
end

initial_population (generic function with 6 methods)

### Auxiliary functions

The `Evolutionary` optimization algorithms are designed work with a individual population represented as a collection of numerical vectors.
- The optimization objective is kept as a `NonDifferentiable` object works with any numerical vector type. This object allows to keep a minimizer value, an objective function and its value for minimizer.

In [13]:
NonDifferentiable(sum, rand(5))

NonDifferentiable{Float64, Vector{Float64}}(sum, 0.0, [NaN, NaN, NaN, NaN, NaN], [0])

However, this type doesn't have a constructor for working with an MLP object, that is of `Flux.Chain` type.

Let's define this missing constructor.

In [15]:
import Evolutionary.NonDifferentiable
NonDifferentiable(f, x::Chain) = NonDifferentiable{Real,typeof(x)}(f, f(x), deepcopy(x), [0,])

NonDifferentiable

Additionaly, we are required to make a copy of individuals, but `Flux` doesn't provide a `copy` & `copyto!` functions for `Chain` objects, only `deepcopy`. We are going to define some missing functions.
- Test if your individual object successully makes a copy using `copy` and `copyto!` functions

In [17]:
import Base: copy, copyto!

copy(ch::Chain) = deepcopy(ch)

function copyto!(l1::Dense{T}, l2::Dense{T}) where {T}
    copyto!(l1.W, l2.W)
    copyto!(l1.b, l2.b)
    l1
end

function copyto!(ch1::Chain, ch2::Chain)
    for i in 1:length(ch1.layers)
        copyto!(ch1.layers[i],ch2.layers[i])
    end
    ch1
end

copyto! (generic function with 134 methods)

### Optimization

Now, we define the parameters of our evolutionary optimization.

In [36]:
opts = Evolutionary.Options(iterations=10000)

                  abstol = 1.0e-32
                  reltol = 1.0e-32
        successive_f_tol = 10
              iterations = 10000
             store_trace = false
              show_trace = false
              show_every = 1
                callback = nothing


In [34]:
algo = GA(
        selection = rouletteinv,
        mutation =  gaussian_mlp(2.0),
        crossover = uniform_mlp,
        mutationRate = 0.95,
        crossoverRate = 0.95,
        populationSize = 100,
        ε = 0.002
    )

GA(100, 0.95, 0.95, 0.002, Evolutionary.rouletteinv, uniform_mlp, var"#mutation#7"{Evolutionary.var"#mutation#29"{Float64}}(Evolutionary.var"#mutation#29"{Float64}(2.0)))

In [44]:
Random.seed!(2947);
res = Evolutionary.optimize(fitness, model, ES(mutation =gaussian_mlp(2.0),mu = 10,lambda = 70), opts)

LoadError: MethodError: no method matching (::var"#mutation#7"{Evolutionary.var"#mutation#29"{Float64}})(::Chain{Tuple{Dense{typeof(relu), Matrix{Float64}, Vector{Float64}}, Dense{typeof(identity), Matrix{Float64}, Vector{Float64}}}}, ::NoStrategy)
[0mClosest candidates are:
[0m  (::var"#mutation#7")(::T) where T<:Chain at In[10]:3

In [39]:
evomodel= Evolutionary.minimizer(res)
@info "MLP" loss=loss(data, evomodel) accuracy = accuracy(data, evomodel)

┌ Info: MLP
│   loss = 3.4680285804291158
│   accuracy = 0.38
└ @ Main In[39]:2
